In [1]:
import numpy as np
import time
from INST_6050A import INST_6050A
from INST_6626A import INST_6626A

load = INST_6050A(22)
print ("Electronic Load: " + load.identify())
psu = INST_6626A(3)
print ("Supply: " + psu.identify())

Electronic Load: HEWLETT-PACKARD,6050A,0,A.01.01

Supply: HP6626A



In [ ]:
load.disable(1)
psu.disable(3)

In [ ]:
minV = 5
maxV = 15
stepV = 5

minI = 0.1
maxI = 1.5
stepI = 0.2

print("Load testing from {:0.4f}A to {:0.4f}A in {:0.4f}A steps...".format(minI, maxI, stepI))
print("and {:0.4f}V to {:0.4f}V in {:0.4f}V steps...".format(minV, maxV, stepV))
print("")

tests = []
for V in np.arange(minV, maxV+stepV, stepV):
    print("V={:0.4f}".format(V))
    print("I\tPin\tPout\tEff")
    psu.setCurrent(3,2)
    psu.setVoltage(3,V)
    psu.enable(3)
    load.set(1,"CC",0)
    load.enable(1)
    Pins = []
    Pouts = []
    Effs = []
    Is = []
    for I in np.arange(minI, maxI+stepI, stepI):
        load.set(1,"CC",I)
        time.sleep(1)
        Pin = psu.getPower(3)
        Pout = load.getPower(1)
        Eff = Pout/Pin
        Im=load.getCurrent(1)
        print("{:0.4f}\t{:0.4f}\t{:0.4f}\t{:0.4f}".format(Im,Pin,Pout,Eff))
        Pins.append(Pin)
        Pouts.append(Pout)
        Is.append(Im)
        Effs.append(Eff)
    load.disable(1)
    tests.append([V,Is,Pins,Pout,Effs])


Load testing from 0.1000A to 1.5000A in 0.2000A steps...
and 5.0000V to 15.0000V in 10.0000V steps...

V=5.0000
I	Pin	Pout	Eff
0.1070	0.5956	0.5338	0.8963
0.3050	1.5905	1.5165	0.9535
0.5020	2.5835	2.4879	0.9630
0.7000	3.5830	3.4573	0.9649
0.8980	4.5834	4.4200	0.9643
1.1040	5.5800	5.3975	0.9673


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
for t in tests:
    plt.plot(t[1],t[4], '-o', label='{:.4f}V'.format(t[0]))    
plt.legend()
plt.title('Efficiency')
plt.xlabel('Load Current (A)')
plt.ylabel('Efficiency $P_{out}$/$P_{in}$')
plt.ylim(top=1)
plt.show()